In [55]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [56]:
dataset = pd.read_csv("../data/pca/pca_fingerprint_results.csv")
dataset

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,class
0,10.996596,-13.581953,-8.045430,-7.038271,-0.383731,-3.875451,-0.990448,1.297883,-0.574683,-1.370185,...,-0.424665,1.330598,-0.871246,1.463499,-0.420871,-0.112825,0.588864,-0.405024,1.157946,1
1,14.380604,-8.148405,-6.970214,-6.883370,-3.731843,2.294834,-2.794362,2.756995,1.141011,4.466174,...,0.104833,2.173534,0.188480,0.616560,1.007843,-0.985466,-1.090612,3.295126,1.582262,1
2,5.898082,-3.955692,-0.753455,0.755042,-5.912044,4.227601,-4.860235,4.010481,0.751071,6.718320,...,-1.402074,2.024921,-4.478798,2.390832,-4.388051,-0.940372,-5.093709,-0.226873,-3.592670,1
3,16.441638,-3.625168,-4.445587,-4.663273,-2.789747,1.248966,-2.006066,1.979731,-1.475145,0.365042,...,-1.109404,1.535672,-0.656872,-0.573160,-0.556370,-0.678389,0.096279,1.865521,0.709983,1
4,15.227068,-3.499011,-7.350787,-2.818675,-1.694236,-2.790198,-1.907673,1.458892,-2.542744,0.429509,...,-0.822749,0.278967,-0.629833,-0.808339,-0.787193,-1.012290,-0.291861,1.405601,0.632089,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7791,-6.519184,-1.975179,1.934837,1.355185,-1.104766,-1.073995,0.635247,-2.174292,0.351231,2.158437,...,0.629990,-0.609683,0.000688,0.777685,0.717557,1.148067,0.361464,-0.174682,-0.640264,2
7792,-8.936481,-3.827097,0.618619,0.926596,-0.369385,-0.559415,2.782636,-5.674792,6.605094,0.775494,...,-0.228395,0.183583,1.194140,-0.755381,-0.534693,0.027072,0.581102,-0.071217,0.163577,2
7793,-10.134237,-5.339714,-0.482419,-0.843599,-0.010870,0.205482,0.433432,-1.170274,1.914583,0.203546,...,-0.941713,0.270500,2.435655,-1.984606,-1.275028,-0.320185,0.439532,0.378725,0.637877,2
7794,-8.829808,-3.759077,0.602438,0.875080,-0.354298,-0.536431,2.552713,-5.185598,5.912753,0.736547,...,-0.726559,0.360763,2.146776,-1.507715,-0.910828,0.172310,0.735103,-0.028415,0.474641,2


In [57]:
dataset["class"].value_counts()

2    5396
0    2322
1      78
Name: class, dtype: int64

# 1. Split Data between Ligand and Decoy compounds

In [58]:
ligand_df = dataset[dataset["class"] == 1].reset_index(drop=True)
decoy_df = dataset[dataset["class"] == 0].reset_index(drop=True)

In [59]:
ligand_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,class
0,10.996596,-13.581953,-8.045430,-7.038271,-0.383731,-3.875451,-0.990448,1.297883,-0.574683,-1.370185,...,-0.424665,1.330598,-0.871246,1.463499,-0.420871,-0.112825,0.588864,-0.405024,1.157946,1
1,14.380604,-8.148405,-6.970214,-6.883370,-3.731843,2.294834,-2.794362,2.756995,1.141011,4.466174,...,0.104833,2.173534,0.188480,0.616560,1.007843,-0.985466,-1.090612,3.295126,1.582262,1
2,5.898082,-3.955692,-0.753455,0.755042,-5.912044,4.227601,-4.860235,4.010481,0.751071,6.718320,...,-1.402074,2.024921,-4.478798,2.390832,-4.388051,-0.940372,-5.093709,-0.226873,-3.592670,1
3,16.441638,-3.625168,-4.445587,-4.663273,-2.789747,1.248966,-2.006066,1.979731,-1.475145,0.365042,...,-1.109404,1.535672,-0.656872,-0.573160,-0.556370,-0.678389,0.096279,1.865521,0.709983,1
4,15.227068,-3.499011,-7.350787,-2.818675,-1.694236,-2.790198,-1.907673,1.458892,-2.542744,0.429509,...,-0.822749,0.278967,-0.629833,-0.808339,-0.787193,-1.012290,-0.291861,1.405601,0.632089,1
5,17.130025,-3.808420,-4.924050,-5.212040,-3.039766,1.531539,-2.384644,2.428395,-1.377488,1.411122,...,-0.760231,1.957892,-0.244268,0.139777,-0.602629,-1.239124,0.537542,2.054770,-0.283153,1
6,17.285471,-3.714313,-4.792839,-5.279551,-3.026864,1.427829,-2.336943,2.374500,-1.547378,1.280448,...,-0.687859,1.833940,-0.232216,-0.011662,-0.657755,-1.365048,0.406031,2.046955,-0.238376,1
7,4.140540,2.339490,0.647358,3.806197,-2.496182,-0.632273,-2.412149,1.656912,-4.781378,2.059584,...,-0.535720,0.924843,0.020154,-0.934424,-0.212801,-0.121974,-0.614628,-0.714652,-0.746142,1
8,12.130178,-7.446414,-7.820945,-3.335555,-2.635230,-2.790517,-2.688600,1.825950,-0.187694,1.658327,...,-0.147585,0.812843,-0.513583,-0.009741,0.147273,-0.222888,-0.450167,0.563955,1.041704,1
9,19.761315,-6.560360,-11.011217,-8.690976,-4.372912,5.369039,-2.634858,4.442513,0.517369,5.069157,...,-0.127951,0.561762,0.236171,2.829602,-0.185255,0.670708,2.822490,-1.081907,1.582238,1


In [60]:
decoy_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,class
0,4.744462,-5.977049,-0.426462,6.875258,-4.944770,-0.081321,-3.388374,0.927046,0.149188,3.490263,...,-0.841939,-0.227289,-0.600790,0.881211,-0.815057,-0.257304,0.015659,-0.674682,0.787644,0
1,18.616656,-4.090988,-3.886856,-4.974335,3.391845,-3.123838,4.098292,-3.849454,-1.677661,2.203731,...,0.204354,-0.824051,0.509214,-0.993434,-0.302164,0.320085,-0.116181,0.826341,0.842240,0
2,-0.426720,-3.413793,-0.179881,2.605793,-1.900717,0.067323,-1.095765,-0.326148,2.884739,2.881889,...,1.641553,0.926994,0.199114,0.115075,-0.787457,0.448487,0.411019,1.884834,-0.346280,0
3,14.678645,-8.750820,-7.167093,-7.935760,3.052605,-3.229477,6.865956,-7.634898,3.610064,6.255011,...,0.083290,-1.207004,1.077431,1.332146,1.404884,-0.400601,0.884124,-0.884929,1.484342,0
4,9.593584,-8.638656,3.051121,-2.899846,-5.661458,11.337477,-1.534611,1.312930,3.814845,3.532463,...,-0.659130,-0.137506,-0.142194,2.452162,-1.227538,1.445466,0.590282,0.174918,1.519953,0
5,18.728722,-5.408195,-0.869281,-8.146121,1.273378,1.121468,4.051868,-4.055572,1.517462,5.561896,...,1.877143,0.393682,-0.266504,-3.336437,1.344660,0.728245,-1.265665,-1.495014,-0.315329,0
6,17.439171,1.083316,-3.187567,-5.831439,2.344051,-3.442174,1.808126,-2.052169,-0.694774,3.994282,...,-0.316428,1.091043,0.358581,0.363825,-0.123477,-0.087290,-1.313585,-0.338700,0.991103,0
7,19.384751,-3.886428,-0.913879,-7.904117,0.476356,-0.145533,2.025449,-2.146869,-0.776525,-4.475033,...,1.156903,0.530610,0.095308,-1.611730,0.646496,-0.052421,-0.001889,-0.719275,-2.319058,0
8,10.429442,-2.717079,6.280579,12.256169,11.013629,-1.207916,-10.373586,3.687788,-5.033247,-3.989095,...,-0.199597,-0.761466,0.487847,-1.508008,-0.032794,-0.702647,-0.542907,2.001569,-0.279164,0
9,1.684616,-5.248740,-0.357764,4.604446,-4.027288,0.880609,-2.503621,-0.071119,2.089897,4.572006,...,0.085649,-0.641270,-1.292863,-0.793326,-0.512335,0.338810,-0.464532,0.654238,0.639348,0


In [61]:
ligand_df.info() # have 78 entries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Columns: 196 entries, 0 to class
dtypes: float64(195), int64(1)
memory usage: 119.6 KB


In [62]:
decoy_df.info() # have 2322 entries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2322 entries, 0 to 2321
Columns: 196 entries, 0 to class
dtypes: float64(195), int64(1)
memory usage: 3.5 MB


# 2. Take 150 random samples of decoy compounds

In [63]:
decoy_samples = decoy_df.sample(n=150, random_state=42).reset_index(drop=True) # Take 150 random samples of decoy compound
ligand_samples = ligand_df.copy() # Take all 78 samples of ligand compound

In [64]:
decoy_samples

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,class
0,4.022761,-7.060943,-3.122727,6.706148,-5.166049,2.301548,-4.583194,4.432560,-4.155426,4.895489,...,-0.298978,-0.396728,1.956584,-1.784317,1.110232,0.604141,0.569589,0.286480,-1.051825,0
1,10.896416,12.242884,2.971191,-8.791574,-2.054758,-7.072289,-1.582277,0.153548,1.516926,3.489578,...,-1.235915,0.750631,-1.403482,-0.555177,-0.836000,-0.245472,-1.314245,-1.002541,-0.369105,0
2,10.101060,0.392941,-0.958007,3.922657,0.747816,-3.300391,4.545585,-5.631004,-2.212439,-5.846624,...,0.820384,1.023873,0.108749,0.180126,-0.412965,0.555189,-0.880650,-0.014968,-0.332649,0
3,16.245734,8.882902,3.587902,-9.320039,-4.167340,0.317424,0.866514,-1.855077,4.270091,-5.082882,...,2.755104,1.037023,-1.773851,-1.265181,0.506058,-2.489170,-2.095464,-1.139908,0.535062,0
4,5.090924,-6.034171,5.765915,0.394056,-4.086182,11.122112,0.679368,-0.860469,1.967793,2.583814,...,0.196037,1.784845,-0.672810,0.759736,0.971426,-1.156872,-0.622250,1.460654,-0.008761,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,15.943941,-5.461339,-6.233845,-0.436273,1.136955,-1.336595,1.647553,-1.387635,-6.393353,0.066315,...,-0.531587,0.874141,-0.847158,0.170383,1.593155,-0.140011,-0.441687,0.599789,1.170091,0
146,0.657123,-7.094526,-1.172757,5.036346,-2.341929,-0.275144,-1.536190,1.149392,-4.907823,-0.232557,...,-0.838162,0.700224,0.815148,-0.253950,0.314200,-0.179370,0.027711,-0.606726,-1.115513,0
147,21.524152,-5.097441,-0.094600,-3.312133,0.476221,2.303059,1.829910,0.748952,-1.586688,-0.934729,...,0.215915,-3.979220,1.589597,-0.990663,-0.423824,-1.072679,-0.496208,0.082637,-1.622708,0
148,22.903416,0.970284,2.464822,-16.137803,-0.669203,5.758147,2.722538,0.000798,0.988748,7.655305,...,-0.832831,1.133788,0.676448,-0.190277,1.231683,2.425419,1.583251,1.041489,3.285936,0


In [65]:
ligand_samples

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,class
0,10.996596,-13.581953,-8.045430,-7.038271,-0.383731,-3.875451,-0.990448,1.297883,-0.574683,-1.370185,...,-0.424665,1.330598,-0.871246,1.463499,-0.420871,-0.112825,0.588864,-0.405024,1.157946,1
1,14.380604,-8.148405,-6.970214,-6.883370,-3.731843,2.294834,-2.794362,2.756995,1.141011,4.466174,...,0.104833,2.173534,0.188480,0.616560,1.007843,-0.985466,-1.090612,3.295126,1.582262,1
2,5.898082,-3.955692,-0.753455,0.755042,-5.912044,4.227601,-4.860235,4.010481,0.751071,6.718320,...,-1.402074,2.024921,-4.478798,2.390832,-4.388051,-0.940372,-5.093709,-0.226873,-3.592670,1
3,16.441638,-3.625168,-4.445587,-4.663273,-2.789747,1.248966,-2.006066,1.979731,-1.475145,0.365042,...,-1.109404,1.535672,-0.656872,-0.573160,-0.556370,-0.678389,0.096279,1.865521,0.709983,1
4,15.227068,-3.499011,-7.350787,-2.818675,-1.694236,-2.790198,-1.907673,1.458892,-2.542744,0.429509,...,-0.822749,0.278967,-0.629833,-0.808339,-0.787193,-1.012290,-0.291861,1.405601,0.632089,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,14.066839,-6.491627,-2.091173,-2.528079,-2.298534,-4.824863,-2.297510,0.239755,2.397044,0.689745,...,-0.294248,-0.749754,-1.624025,0.233379,1.680851,0.573061,-0.575019,0.781579,-2.756030,1
74,15.599291,6.118581,-0.615089,-9.278798,-1.751151,-3.066809,0.152417,-1.444177,4.582531,-2.683279,...,-1.204551,0.991233,0.629331,0.908100,0.701706,0.135488,-0.173850,-0.300493,0.348620,1
75,5.874727,14.787542,6.059764,-2.985573,-6.662480,-1.436483,-4.851312,2.747031,0.743673,0.513223,...,0.576287,0.130856,0.746882,-1.025057,-0.366139,0.195104,-0.208159,-0.252302,0.716349,1
76,9.107021,-1.187344,-0.628291,6.129179,-2.053594,-0.193606,-2.227678,2.275277,-7.771191,0.467247,...,0.669240,-0.410826,-1.060554,-0.011827,0.791731,0.072175,-0.240811,1.110101,-1.165583,1


# 3. Concatenate data of randomly sampled Decoy with Ligand

In [66]:
sampled_data = pd.concat([decoy_samples, ligand_samples], axis=0).reset_index(drop=True)
sampled_data.head(10)

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,class
0,4.022761,-7.060943,-3.122727,6.706148,-5.166049,2.301548,-4.583194,4.432560,-4.155426,4.895489,...,-0.298978,-0.396728,1.956584,-1.784317,1.110232,0.604141,0.569589,0.286480,-1.051825,0
1,10.896416,12.242884,2.971191,-8.791574,-2.054758,-7.072289,-1.582277,0.153548,1.516926,3.489578,...,-1.235915,0.750631,-1.403482,-0.555177,-0.836000,-0.245472,-1.314245,-1.002541,-0.369105,0
2,10.101060,0.392941,-0.958007,3.922657,0.747816,-3.300391,4.545585,-5.631004,-2.212439,-5.846624,...,0.820384,1.023873,0.108749,0.180126,-0.412965,0.555189,-0.880650,-0.014968,-0.332649,0
3,16.245734,8.882902,3.587902,-9.320039,-4.167340,0.317424,0.866514,-1.855077,4.270091,-5.082882,...,2.755104,1.037023,-1.773851,-1.265181,0.506058,-2.489170,-2.095464,-1.139908,0.535062,0
4,5.090924,-6.034171,5.765915,0.394056,-4.086182,11.122112,0.679368,-0.860469,1.967793,2.583814,...,0.196037,1.784845,-0.672810,0.759736,0.971426,-1.156872,-0.622250,1.460654,-0.008761,0
5,13.200631,-6.584079,-6.912367,-3.610110,-1.902176,-2.730864,-1.271967,0.254899,-0.699497,2.545727,...,-0.211441,0.934955,-0.026601,0.493040,0.014791,0.289944,0.585126,-1.288744,-0.146990,0
6,6.568622,1.366820,3.781401,4.076924,4.457311,-2.724677,-4.192927,-0.964424,-4.607617,0.878173,...,2.085779,-0.219197,-1.502176,5.865008,-1.873118,-2.467885,0.021170,1.117391,1.707667,0
7,8.472444,-8.627165,9.914423,-7.386450,-5.237825,12.965056,-1.728055,3.895913,4.683675,4.038980,...,-0.163817,-2.830403,0.107816,-1.561394,-0.953352,-0.757268,-0.581223,-0.031787,-1.201532,0
8,14.058249,-8.304185,0.556353,-11.200058,-0.067454,2.430836,4.800424,-4.776698,5.592344,9.675994,...,-0.830896,1.000233,0.191471,-1.841646,2.722601,-0.989469,-0.989466,2.519952,-0.416639,0
9,5.903162,0.820961,-1.618191,0.620798,-3.069442,0.746703,-3.654782,3.198979,-2.375475,5.860568,...,-1.567403,0.785349,-0.196636,-0.432670,1.276068,0.649963,0.676486,-1.002508,1.056771,0


In [67]:
sampled_data.info() # Have 228 entries, 150 of decoys and 78 of ligands

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Columns: 196 entries, 0 to class
dtypes: float64(195), int64(1)
memory usage: 349.3 KB


# 4. Oversampling of Ligand using SMOTE

In [68]:
# Separate features and labels
X = sampled_data.drop(columns=['class'])
y = sampled_data['class']

In [69]:
# Initiate SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=7)

# Do oversampling
X_resampled, y_resampled = smote.fit_resample(X, y)

In [70]:
# Create a new DataFrame with the oversampled data
oversampled_data = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled, name='class')], axis=1)
oversampled_data

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,class
0,4.022761,-7.060943,-3.122727,6.706148,-5.166049,2.301548,-4.583194,4.432560,-4.155426,4.895489,...,-0.298978,-0.396728,1.956584,-1.784317,1.110232,0.604141,0.569589,0.286480,-1.051825,0
1,10.896416,12.242884,2.971191,-8.791574,-2.054758,-7.072289,-1.582277,0.153548,1.516926,3.489578,...,-1.235915,0.750631,-1.403482,-0.555177,-0.836000,-0.245472,-1.314245,-1.002541,-0.369105,0
2,10.101060,0.392941,-0.958007,3.922657,0.747816,-3.300391,4.545585,-5.631004,-2.212439,-5.846624,...,0.820384,1.023873,0.108749,0.180126,-0.412965,0.555189,-0.880650,-0.014968,-0.332649,0
3,16.245734,8.882902,3.587902,-9.320039,-4.167340,0.317424,0.866514,-1.855077,4.270091,-5.082882,...,2.755104,1.037023,-1.773851,-1.265181,0.506058,-2.489170,-2.095464,-1.139908,0.535062,0
4,5.090924,-6.034171,5.765915,0.394056,-4.086182,11.122112,0.679368,-0.860469,1.967793,2.583814,...,0.196037,1.784845,-0.672810,0.759736,0.971426,-1.156872,-0.622250,1.460654,-0.008761,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,5.985160,12.779421,3.230932,-1.558969,-6.045938,0.241865,-5.473675,4.361625,-0.249197,1.695995,...,-0.915841,0.476160,-0.158109,-0.985972,-0.777564,-0.300246,0.151411,-0.393583,-0.297312,1
296,12.871567,-5.836833,-8.493318,6.386544,-3.031352,1.297841,-2.587385,2.111745,-3.450812,-1.700955,...,-0.012749,0.617378,-0.853879,2.227534,1.025873,-0.638069,-0.183974,-0.059744,-0.861809,1
297,5.724934,13.276735,-0.890055,-2.016508,-7.918346,-5.399766,-10.249366,7.243133,1.054543,2.582620,...,-0.064411,-0.641454,-0.412888,0.643862,-1.093087,0.250658,0.393813,-0.754105,-1.153816,1
298,16.751409,-4.134484,-5.920277,-2.643875,2.578754,-3.481571,-1.590502,-1.337279,-1.119437,-1.102343,...,-0.149072,0.616654,-0.908813,-1.023145,-0.441327,-1.077154,-0.666029,1.207927,-0.274054,1


In [71]:
oversampled_data.info() # have 300 entries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Columns: 196 entries, 0 to class
dtypes: float64(195), int64(1)
memory usage: 459.5 KB


In [72]:
oversampled_data["class"].value_counts() # Balanced data between ligand and decoy

0    150
1    150
Name: class, dtype: int64

In [73]:
oversampled_data.to_csv("../data/resampled/oversampled_data.csv", index=False)

# 5. Undersampling of Decoy using RandomRandomSampler

In [74]:
# Initiate undersampler
undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=3)

# Do undersamping
X_resampled, y_resampled = undersampler.fit_resample(X, y)

In [75]:
# Create a new DataFrame with the undersampled data
undersampled_data = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled, name='class')], axis=1).reset_index(drop=True)
undersampled_data.head(10)

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,class
0,16.306559,-4.053504,11.544170,-2.914599,-4.030473,20.299897,6.806870,-1.898586,-2.380656,-11.175384,...,0.352263,0.455304,1.484919,3.270641,-0.243741,-1.654044,-2.346995,-2.046216,-3.219721,0
1,16.245734,8.882902,3.587902,-9.320039,-4.167340,0.317424,0.866514,-1.855077,4.270091,-5.082882,...,2.755104,1.037023,-1.773851,-1.265181,0.506058,-2.489170,-2.095464,-1.139908,0.535062,0
2,10.949707,-6.528771,4.805917,-1.687464,-3.369688,12.780447,0.578050,-0.360049,-1.469986,1.376761,...,0.178499,0.815886,1.957061,0.114077,-0.336606,1.682449,-0.734458,1.201305,-0.321612,0
3,8.449733,-7.293462,-3.541505,-1.948532,-2.576560,-3.943999,0.800536,-3.993533,6.958001,2.386961,...,-0.596081,-0.773240,-1.001076,-0.323051,-0.809206,-0.824705,-0.620043,0.388324,-0.381711,0
4,20.611143,-2.472154,4.695242,-17.555197,-0.966808,8.507590,2.340409,0.683153,2.331995,4.358220,...,0.365272,1.789154,1.105863,-0.033007,0.678282,0.970452,-0.031796,-1.639882,0.221858,0
5,4.841889,-6.117708,6.120161,1.314364,-4.935512,10.519976,1.484332,-3.070978,5.477355,2.984986,...,0.053382,1.311598,-1.222493,1.290492,0.644353,-1.010501,0.328273,1.121712,-0.182285,0
6,15.943941,-5.461339,-6.233845,-0.436273,1.136955,-1.336595,1.647553,-1.387635,-6.393353,0.066315,...,-0.531587,0.874141,-0.847158,0.170383,1.593155,-0.140011,-0.441687,0.599789,1.170091,0
7,6.568622,1.366820,3.781401,4.076924,4.457311,-2.724677,-4.192927,-0.964424,-4.607617,0.878173,...,2.085779,-0.219197,-1.502176,5.865008,-1.873118,-2.467885,0.021170,1.117391,1.707667,0
8,9.433447,0.739057,3.432587,9.317039,-0.920094,-0.897364,0.397768,0.031468,-1.280686,-3.600888,...,0.463526,-0.346860,0.288047,0.535187,1.622070,-1.669075,-0.012705,0.324189,0.838047,0
9,16.448667,-1.616194,5.496420,0.129803,3.846117,5.393916,2.511988,-6.323913,-0.614575,-8.720116,...,1.118596,0.446822,1.244813,-1.253294,0.334098,0.971675,-0.333229,0.952757,0.566563,0


In [76]:
undersampled_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Columns: 196 entries, 0 to class
dtypes: float64(195), int64(1)
memory usage: 239.0 KB


In [77]:
undersampled_data["class"].value_counts() # Balanced data between ligand and decoy

0    78
1    78
Name: class, dtype: int64

In [78]:
undersampled_data.to_csv("../data/resampled/undersampled_data.csv", index=False)